In [1]:
#meta 12/12/2020 Kaggle Cassava Model - Inference
#fast.ai 2020 study group Kaggle Competition
#src https://www.kaggle.com/c/cassava-leaf-disease-classification
#prev competition src https://www.kaggle.com/c/cassava-disease
#references https://arxiv.org/pdf/1908.02900.pdf

#input: df_train.pkl, export.pkl
#output: submission.csv
#pwd: /kaggle/working

#history
#here 1/10/2020 INFERENCE - ALPHA MODEL (POC)
#      Export saved alpha dataframe & model and create submission.csv file
#      Alpha model - manually tuned model
#      POC export a model and make submissions.
#      Successful submit with ? accuracy

#was 1/9/2020 MODEL - BACK TO v15 SCORE0.877, TRAIN FT10. EXPORT FOR INFERENCE
#      in FULL mode, train entire ds
#      dataprep: dblocksplitter=ColSplitter(), item_tfms = CropPad(500), batch_tfms = aug_transforms(mult=2))
#      model: learn = cnn_learner(dls, resnet18, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)
#      learn.fine_tune(10)
#      export output for inference; no submission file output
# 1/11/2020 MODEL - SAME AS ABOVE -> MINI, TRAIN FT1. EXPORT FOR INFERENCE
#      MINI exports, trained on 50% subsample 
#      Mini model, test for inference on GPU during submission. Score .139
# 1/11/2020 MODEL - SAME AS ABOVE -> EXPORT FULL DATA AND MODEL FOR INFERENCE
#      FULL exports, trained on 100% train ds
#      Full model, test for inference on GPU during submission. Score .139
# 1/11/2020 MODEL - SAME AS ABOVE -> NO GPU. Score .139
# 1/11/2020 MODEL - SAME AS ABOVE -> NO GPU. Added loop for test_img. Score .139
#      Error: dls is not defined (because of CPU)
# 1/11/2020 MODEL - SAME AS ABOVE -> GPU. Score .139
#      RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

#1/11/2020 MODEL - BACK TO v0 INFERENCE ONLY SUCCESSFUL SUBMIT. SCORE 0.652. ALPHA EXPORTS.
#      FULL exports, trained on 100% train ds
#      Full model?, alpha?, test for inference on CPU during submission. Same Score 0.652

#1/12/2020 MODEL - BACK TO SUCCESSFUL v0 INFERENCE. SAME SCORE0.652. LATEST EXPORTS.
#      FULL exports, trained on 100% train ds
#      Full model?, switched from alpha to latest?, test for inference on CPU. Score 0.747
#      Track inference time .1376 min but it really took at least an hour.
#1/12/2020 MODEL - SAME AS ABOVE, ON GPU
#      RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

#1/12/2020 MODEL - INFERENCE (CPU). ADD TTA.
#      Same as above, add TTA. Score ?

#here 1/12/2020 MODEL - POC INFERENCE (CPU): REDO TEST IMAGES DATA ONLY. NO TTA FOR POC.
#      Same as above, add DEBUG for testing mini model related exports. 
#      Redo test images data only -> no need for exported data, no need for dls with train data
#      note: for now, previous code commented out.  if POC submission works, remove unnecessary code.
#      Score ?

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import os
#import time #to track performance time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#modeling
from fastai.vision.all import *

# Input data files are available in the read-only "../input/" directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#--constants and variables
#SEED = 24
DEBUG = 1

#set paths
path = Path('../input/cassava-leaf-disease-classification')
if not DEBUG: 
    path_model = Path('../input/my-cassava-model')
    #path_data = Path('../input/my-cassava-data')
else:
    path_model = Path('../input/my-cassava-mini')
    #path_data = Path('../input/my-cassava-mini')

#--functions
def get_x(row): return path/'train_images'/row['image_id']
def get_y(row): return row['label']


# Cassava Disease Classification - Inference
##### Objective: Classify pictures of cassava leaves into 1 of 4 disease categories or healthy.

Export saved data & model and create a submission.csv file, with predictions on test images.


## 0. Load Prepared Data and Model

- Input(s): df_train.pkl and export.pkl


In [4]:
#track time
t0 = time.time()

'''$delete
#load prepared dataframe
df_train = torch.load(path_data/'df_train.pkl')

#dataloaders
dblock = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_x = get_x,
                 get_y = get_y,
                 splitter=ColSplitter(),
                 item_tfms = CropPad(500), batch_tfms = aug_transforms(mult=2))

dls = dblock.dataloaders(df_train, bs=64)
'''

"$delete\n#load prepared dataframe\ndf_train = torch.load(path_data/'df_train.pkl')\n\n#dataloaders\ndblock = DataBlock(blocks = (ImageBlock, CategoryBlock),\n                 get_x = get_x,\n                 get_y = get_y,\n                 splitter=ColSplitter(),\n                 item_tfms = CropPad(500), batch_tfms = aug_transforms(mult=2))\n\ndls = dblock.dataloaders(df_train, bs=64)\n"

In [5]:
#check if loaded model exists and load it
if path_model.ls(file_exts='.pkl'):
    print(path_model)
    learn_inf = load_learner(path_model/'export.pkl')
else:
    print("No model to export")

../input/my-cassava-mini


### 0.1 Images  
1 test image

In [6]:
'''#load test images
test_img = get_image_files(path/'test_images')

print("Number of files: ", len(test_img))
'''

'#load test images\ntest_img = get_image_files(path/\'test_images\')\n\nprint("Number of files: ", len(test_img))\n'

## 3. Predict
Predict on one file

In [7]:
#view test images - only one
#Image.open(test_img[0])

#predict one, returns a tuple, get 0th item
#y_hat = learn_inf.predict(test_img[0])
#y_hat[0]

Test dataset

In [8]:
#template
sample_submission = pd.read_csv(path/'sample_submission.csv')
submission = sample_submission.copy()


In [9]:
#create test images id
'''test_img_id = []

[test_img_id.append(f) for f in test_img]    
'''
test_img_id = [os.path.join(path/'test_images', x) for x in submission['image_id'].values]

#images in test files
test_img_id

['../input/cassava-leaf-disease-classification/test_images/2216849948.jpg']

In [10]:
#test_dl = dls.test_dl(test_img_id)
test_dl = learn_inf.dls.test_dl(test_img_id)

len(test_dl)

1

## 4. Submit


In [11]:
#template
#sample_submission = pd.read_csv(path/'sample_submission.csv')
#submission = sample_submission.copy()

#predict
preds,_ = learn_inf.get_preds(dl=test_dl)
#preds, _ = learn_inf.tta(dl=test_dl, n=5, beta=0) #n=15
#preds

#submission
#submission['image_id'] = [ str(item).split("/")[-1] for item in test_img_id]
submission['label'] = preds.argmax(dim=-1).numpy()
submission.head()


,image_id,label
0,2216849948.jpg,4


In [12]:
#save to csv
submission.to_csv('submission.csv', index=False)

#total inference time
t1 = time.time()
print ("Inference time (in min)", (time.time() - t0)/60)

Inference time (in min) 0.055401893456776936


## Xtra

In [13]:
#$xtra
#define label lookup 
#def lookup_label(image_id): return int(df_train[df_train['image_id']== image_id]['label'])

#test function
#lookup_label('100042118.jpg')